# 🌐 Step 05: RAG 웹 애플리케이션 구현

이 노트북에서는 이전에 만든 RAG 시스템을 웹 애플리케이션으로 구현합니다.
Streamlit을 사용하여 사용자 친화적인 인터페이스를 만들고,
실시간으로 질문하고 답변을 받을 수 있는 시스템을 구축합니다.

## 📚 1단계: 필요한 라이브러리 설치 및 임포트

Streamlit 관련 패키지를 추가로 설치하고 필요한 라이브러리들을 임포트합니다.
- streamlit: 웹 애플리케이션 구축
- streamlit-chat: 채팅 인터페이스 구현

In [ ]:
# 필요한 패키지 설치
!pip install streamlit==1.31.0 streamlit-chat==0.1.1

## 📝 2단계: 웹 애플리케이션 코드 작성

Streamlit 애플리케이션 코드를 작성합니다.
이 코드는 별도의 Python 파일로 저장되어 실행됩니다.

In [ ]:
%%writefile ../app/rag_app.py

import os
import json
import streamlit as st
from streamlit_chat import message
from dotenv import load_dotenv
import pandas as pd
from elasticsearch import Elasticsearch
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import ElasticsearchStore
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# 페이지 설정
st.set_page_config(
    page_title="데이터베이스 메타데이터 도우미",
    page_icon="🔍",
    layout="wide"
)

# 제목 표시
st.title("🤖 데이터베이스 메타데이터 도우미")
st.markdown("""
이 애플리케이션은 데이터베이스 메타데이터에 대한 질문에 답변해드립니다.
테이블 구조, 컬럼 설명, 데이터 특성 등에 대해 자유롭게 질문해보세요!
""")

@st.cache_resource
def initialize_rag_system():
    """RAG 시스템 초기화"""
    # 환경 변수 로드
    load_dotenv()
    
    # Elasticsearch 연결
    es = Elasticsearch(
        os.getenv('ELASTICSEARCH_URL'),
        basic_auth=(
            os.getenv('ELASTICSEARCH_USERNAME'),
            os.getenv('ELASTICSEARCH_PASSWORD')
        ),
        verify_certs=False
    )
    
    # 임베딩 모델 초기화
    embeddings = OpenAIEmbeddings()
    
    # 벡터 저장소 연결
    vector_store = ElasticsearchStore(
        es_connection=es,
        index_name="metadata-rag",
        embedding=embeddings
    )
    
    # ChatGPT 모델 초기화
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    
    # RAG 체인 생성
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever()
    )
    
    return qa_chain

# RAG 시스템 초기화
qa_chain = initialize_rag_system()

# 세션 상태 초기화
if 'messages' not in st.session_state:
    st.session_state.messages = []

# 채팅 히스토리 표시
for i, (role, content) in enumerate(st.session_state.messages):
    message(content, is_user=(role == "user"), key=str(i))

# 사용자 입력 처리
if prompt := st.chat_input("질문을 입력하세요..."):
    # 사용자 메시지 추가
    st.session_state.messages.append(("user", prompt))
    message(prompt, is_user=True)
    
    # RAG 시스템으로 답변 생성
    response = qa_chain.run(prompt)
    
    # 답변 메시지 추가
    st.session_state.messages.append(("assistant", response))
    message(response, is_user=False)

# 사이드바에 도움말 추가
with st.sidebar:
    st.markdown("### 💡 질문 예시")
    st.markdown("""
    - customer_orders 테이블의 구조가 어떻게 되나요?
    - 주문 금액의 범위는 어떻게 되나요?
    - 주문 상태는 어떤 값들이 가능한가요?
    - 고객의 이메일 정보는 어디에 저장되나요?
    """)
    
    st.markdown("### ℹ️ 사용 가능한 테이블")
    st.markdown("""
    - customer_orders: 주문 정보
    - customers: 고객 정보
    """)

## 🚀 3단계: 웹 애플리케이션 실행

작성한 Streamlit 애플리케이션을 실행합니다.
실행하면 웹 브라우저에서 대화형 인터페이스를 통해
RAG 시스템과 상호작용할 수 있습니다.

In [ ]:
# 애플리케이션 실행
!streamlit run ../app/rag_app.py

## 🎨 웹 애플리케이션 주요 기능

1. 사용자 인터페이스
   - 깔끔하고 직관적인 채팅 인터페이스
   - 실시간 질의응답
   - 채팅 히스토리 유지

2. RAG 시스템 통합
   - 이전 단계에서 구현한 RAG 시스템 활용
   - 메타데이터 기반 정확한 답변 제공
   - 빠른 응답 속도

3. 도움말 기능
   - 질문 예시 제공
   - 사용 가능한 테이블 목록
   - 직관적인 사이드바 메뉴

## 🔍 사용 방법

1. 웹 브라우저에서 애플리케이션 접속
2. 채팅창에 질문 입력
3. 실시간으로 답변 확인
4. 사이드바의 예시 참고

이제 누구나 쉽게 데이터베이스 메타데이터를 조회하고
이해할 수 있는 인터페이스가 준비되었습니다!